In [21]:
import cv2 #pip install opencv-python
import pandas as pd #pip install pandas
import os #no need to install
from pyzbar.pyzbar import decode #pip install pyzbar 
from openpyxl import load_workbook #pip install openpyxl
import numpy as np
import glob

os.chdir("D:\LEARN\Python\Final_Project")

In [30]:
card_list = list()
# cards = [os.path.basename(x) for x in glob.glob("Final_Project\*.jpg")]
for name in glob.glob("Final_Project\*.jpg"): 
  card = cv2.imread(name)
  card_list.append(card)

print(card_list)

[]


### Ex3: Create a student dataset with ID

In [3]:
student_data = pd.read_excel("DATA.xlsx", sheet_name = "Student data")
student_data.head()

,Student_ID,STUDENT,Student_Major,Student_Class,Student_Email
0,20040004,Nguyễn Quang Hào,DS,20DS,hao.nguyen200404@vnuk.edu.vn
1,20050005,Phan Thanh Lâm,THM,20THM2,NaN
2,20010039,Lê Đại Hải,IBM,20IBM3,NaN
3,20040000,Lê Thị Tấm,CSE,20CSE,NaN
4,20040001,Trần Chín,THM,20THM1,NaN


### Ex4: Create a book dataset with code 

In [4]:
book_data = pd.read_excel("DATA.xlsx", sheet_name = "Book data")
book_data['CODE'] = book_data['CODE'].apply(lambda x: '{:.0f}'.format(x))
book_data.head()

,CODE,BOOK,AUTHOR
0,8936037799377,Những Cuộc Phiêu Lưu Trong Kinh Doanh,John Brooks
1,8935251405972,Tư Duy Nhanh Và Chậm,Daniel Kahneman
2,8935235219489,Vào Trong Hoang Dã,Jon Krakauer
3,8935235229693,Tương Lai Sau Đại Dịch Covid,Jason Schenker
4,8935235218130,Yêu những điều không hoàn hảo,Hea Min


### Ex5: Scan infomation on Student Card

In [25]:
card = cv2.imread('Student_Card/STUDENT_CARD_00.jpg')
card_01 = cv2.imread('Student_Card/STUDENT_CARD_01.jpg')
card_02 = cv2.imread('Student_Card/STUDENT_CARD_02.jpg')
def student_card_detect(card):
    code = decode(card)
    id = code[0].data.decode('utf-8')
    for i in range(len(student_data)):
        if int(id) == int(student_data.loc[i,'Student_ID']):
            return(student_data.loc[i])

print(student_card_detect(card)) 


IndexError: list index out of range

### Ex6: Get information on Book

In [6]:
book_01 = cv2.imread("Book_List/BOOK_01.jpg")
book_02 = cv2.imread("Book_List/BOOK_02.jpg")
book_03 = cv2.imread("Book_List/BOOK_03.jpg")

def book_detect(book):
    book_code = decode(book)
    code = book_code[0].data.decode('utf-8')

    for i in range(len(book_data)):
        if code == book_data.loc[i,'CODE']:
            return(book_data.loc[i])
         
# print(book_detect(book_02))
# print(book_info['BOOK'])

### Ex7: Import Book to borrow book list when student get the book

In [7]:
card_info = student_card_detect(card)
card_info_01 = student_card_detect(card_01)
cart_info_02 = student_card_detect(card_02)
book_info = book_detect(book_01)
book_info_02 = book_detect(book_02)
book_info_03 = book_detect(book_03)
borrow_list = pd.DataFrame(columns=['BOOK','ID','Student_name'],)  
def borrow_book(card_info, book_info):
    borrow_info = pd.DataFrame({'BOOK':book_info['BOOK'],'ID':card_info['Student_ID'],'Student_name':card_info['STUDENT']}, index = [0])
    return borrow_info

borrow_list = borrow_list.append(borrow_book(card_info, book_info))
borrow_list = borrow_list.append(borrow_book(card_info, book_info))
borrow_list = borrow_list.append(borrow_book(card_info, book_info))
borrow_list = borrow_list.append(borrow_book(card_info, book_info))


file = pd.ExcelWriter('DATA.xlsx', mode='a', if_sheet_exists = 'replace')
borrow_list.to_excel(file, 'Borrowed data')
file.save()
print(borrow_list)

                           BOOK        ID      Student_name
0            Vào Trong Hoang Dã  20040004  Nguyễn Quang Hào
0  Tương Lai Sau Đại Dịch Covid  20040004  Nguyễn Quang Hào
0          Tư Duy Nhanh Và Chậm  20040004  Nguyễn Quang Hào


In [18]:
#detect clone Student card:
card = cv2.imread('Student_Card/STUDENT_CARD_00_diff.jpg')
card = cv2.resize(card,(600,400))

cardGray = cv2.cvtColor(card,cv2.COLOR_BGR2GRAY)
cardBlur = cv2.GaussianBlur(cardGray,(5,5),10)
cardCanny = cv2.Canny(cardGray,150,200)
countours,__ = cv2.findContours(cardCanny, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
cv2.drawContours(card,countours,-1,(0,255,0),1)

def rectangle_countours(countours):
    rect_cnts = []
    for c in countours:
        area = cv2.contourArea(c)
        # print(area)
        if area > 50:
            peri = cv2.arcLength(c,True)
            approx = cv2.approxPolyDP(c,0.02*peri,True)
            # print(len(approx))
            if len(approx) == 4:
                rect_cnts.append(c)   
    rect_cnts = sorted(rect_cnts,key = cv2.contourArea,reverse=True)
    return rect_cnts

def get_conrner_point(c):
    peri = cv2.arcLength(c,True)
    approx = cv2.approxPolyDP(c,0.04*peri,True)
    return approx

def four_points(points):
    points = points.reshape(4,2)
    new_points = np.zeros((4,1,2),dtype="float32")

    s = points.sum(axis = 1)
    new_points[0] = points[np.argmin(s)]
    new_points[3] = points[np.argmax(s)]

    diff = np.diff(points, axis = 1)
    new_points[1] = points[np.argmin(diff)]
    new_points[2] = points[np.argmax(diff)]

    return new_points

rect_cnts = rectangle_countours(countours)
biggest_countours = get_conrner_point(rect_cnts[0])

if biggest_countours.size != 0:
    # cv2.drawContours(card,biggest_countours,-1,(255,0,0),20)
    # four_points(biggest_countours)    
    biggest_countours = four_points(biggest_countours)

    point_1 = np.float32(biggest_countours)
    point_2 = np.float32([[0,0],[600,0],[0,400],[600,400]])
    
    matrix = cv2.getPerspectiveTransform(point_1,point_2)
    card_warp = cv2.warpPerspective(card,matrix,(600,400))     
               
cv2.imshow('card',card)
cv2.waitKey(0)

IndexError: list index out of range